In [38]:
import xml.etree.ElementTree as et
import pandas as pd
import numpy as np
from datetime import datetime as dt
import csv

In [10]:
file_path = "Match_Day_1-19/f24-24-2016-853"
file_path2 = "-eventdetails.xml"
numeros = range(139,329)
#noms de fichiers file_path+"numero"+file_path2
liste_files = [file_path+str(num)+file_path2 for num in numeros]

In [57]:
def shot_in_game(game_file):
    tree = et.ElementTree(file = game_file)
    games = tree.getroot()
    match_details = games[0].attrib
    
    team_dict = {match_details["home_team_id"]: match_details["home_team_name"],
             match_details["away_team_id"]: match_details["away_team_name"]}
    
    game_id = match_details["id"]
    shot_name = []
    shot_x = []
    shot_y = []
    shot_zone = []
    shot_min = []
    shot_sec = []
    shot_period = []
    shot_team = []
    goalmouth_y = []
    goalmouth_z = []
    saved_x = []
    saved_y = []
    body_part = []
    shot_play = []
    shot_player = []

    shot_dict = {'13': 'Shot off target',
                 '14': 'Post',
                 '15': 'Shot saved',
                 '16': 'Goal'}

    body_dict = {"15": "head",
                "72": "left foot",
                "20": "right foot",
                "21": "other body part"}

    shot_play_dict = {'22': 'regular play',
                '23': 'fast break',
                '24': 'set piece',
                '25': 'from corner',
                '26': 'free kick',
                '96': 'corner situation',
                '112': 'scramble',
                '160': 'throw-in set piece',
                '9': 'penalty',
                '28': 'own goal'}

    for game in games:

        for event in game:

            if event.attrib.get("type_id") in ['13', '14', '16']:

                shot_name.append(shot_dict[event.attrib.get("type_id")])
                shot_x.append(event.attrib.get("x"))
                shot_y.append(event.attrib.get("y"))
                shot_min.append(event.attrib.get("min"))
                shot_sec.append(event.attrib.get("sec"))
                shot_period.append(event.attrib.get("period_id"))
                shot_team.append(team_dict[event.attrib.get("team_id")])
                shot_player.append(event.attrib.get("player_id"))

                for q in event:

                    qualifier = q.attrib.get("qualifier_id")
                    if qualifier == '102':
                        saved_x.append('')
                        saved_y.append('')
                        goalmouth_y.append(q.attrib.get("value"))
                    if qualifier == '103':
                        goalmouth_z.append(q.attrib.get("value"))
                    if qualifier in body_dict.keys():
                        body_part.append(body_dict[qualifier])
                    if qualifier in shot_play_dict.keys():
                        shot_play.append(shot_play_dict[qualifier])

            if event.attrib.get("type_id") == '15':

                shot_name.append(shot_dict[event.attrib.get("type_id")])
                shot_x.append(event.attrib.get("x"))
                shot_y.append(event.attrib.get("y"))
                shot_min.append(event.attrib.get("min"))
                shot_sec.append(event.attrib.get("sec"))
                shot_period.append(event.attrib.get("period_id"))
                shot_team.append(team_dict[event.attrib.get("team_id")])
                shot_player.append(event.attrib.get("player_id"))


                for q in event:

                    qualifier = q.attrib.get("qualifier_id")
                    if qualifier == '146':
                        goalmouth_y.append('')
                        goalmouth_z.append('')
                        saved_x.append(q.attrib.get("value"))
                    if qualifier == '147':
                        saved_y.append(q.attrib.get("value"))
                    if qualifier in ["15", "72", "20", "21"]:
                        body_part.append(body_dict[qualifier])
                    if qualifier in shot_play_dict.keys():
                        shot_play.append(shot_play_dict[qualifier])


    shot_df = zip([game_id for i in range(len(shot_team))],shot_team, shot_player, shot_period, shot_min, shot_sec, shot_play, shot_name, body_part, shot_x, shot_y, 
                           goalmouth_y, goalmouth_z, saved_x, saved_y)

    shot_fieldnames = ["game_id","team", "player", "period", "min", "sec", "shot play", "shot type", "body part", "x", "y", "goalmouth y", 
                       "goalmouth z", "saved x", "saved y"]
    
    df = pd.DataFrame.from_dict(shot_df)
    df.columns = shot_fieldnames
    
    return df

In [58]:
shot_in_game(liste_files[0])

,game_id,team,player,period,min,sec,shot play,shot type,body part,x,y,goalmouth y,goalmouth z,saved x,saved y
0,853139,Nancy,38008,1,5,1,set piece,Shot saved,left foot,80.6,64.8,,,84.6,61.7
1,853139,Nancy,97290,1,5,4,set piece,Shot off target,right foot,75.2,38.8,58.0,20.8,,
2,853139,Nancy,170034,1,11,1,from corner,Shot saved,head,88.8,53.0,,,98.2,50.7
3,853139,Nancy,38008,1,11,3,from corner,Shot off target,right foot,72.4,51.3,58.7,13.9,,
4,853139,Nancy,171101,1,12,50,set piece,Shot off target,head,86.5,47.3,60.7,31.9,,
5,853139,Nancy,9980,1,22,44,regular play,Shot off target,right foot,66.1,35.5,0.2,1.4,,
6,853139,Lyon,59966,1,27,53,free kick,Shot saved,right foot,80.8,62.3,,,99.0,51.8
7,853139,Nancy,51327,1,29,43,set piece,Shot off target,head,90.7,48.9,40,65.3,,
8,853139,Lyon,169007,1,32,3,regular play,Shot saved,right foot,89.7,34.5,,,92.9,37.5
9,853139,Lyon,59966,1,32,5,regular play,Goal,right foot,91.0,32.8,51.5,2.5,,


In [61]:
df = shot_in_game(liste_files[0])
for i in range(1,len(liste_files)):
    df=df.append(shot_in_game(liste_files[i]),ignore_index=True)
df.head()

,game_id,team,player,period,min,sec,shot play,shot type,body part,x,y,goalmouth y,goalmouth z,saved x,saved y
0,853139,Nancy,38008,1,5,1,set piece,Shot saved,left foot,80.6,64.8,,,84.6,61.7
1,853139,Nancy,97290,1,5,4,set piece,Shot off target,right foot,75.2,38.8,58.0,20.8,,
2,853139,Nancy,170034,1,11,1,from corner,Shot saved,head,88.8,53.0,,,98.2,50.7
3,853139,Nancy,38008,1,11,3,from corner,Shot off target,right foot,72.4,51.3,58.7,13.9,,
4,853139,Nancy,171101,1,12,50,set piece,Shot off target,head,86.5,47.3,60.7,31.9,,


In [62]:
df.shape

(4453, 15)